## 1. 개발

In [1]:
%%writefile ../src/timer.sol
pragma solidity 0.6.4;

contract Timer {
    uint256 startTime;
    
    function start() public {
        startTime = now;
    }
    
    function timePassed() public view returns(uint256) {
        return now - startTime;
    }
    
    function getNow() public view returns(uint256) {
        return now;
    }
}

Writing ../src/timer.sol


## 2. 컴파일
**컴파일 결과를 javascript 파일로 저장**
1. node에서 그 파일을 `require`해서 사용하거나
2. geth에서 `loadScript()` 명령어로 실행 가능

In [15]:
!solc --optimize --combined-json abi ../src/timer.sol > ../src/timerABI.json

In [16]:
!solc --optimize --combined-json bin ../src/timer.sol > ../src/timerBIN.json

In [26]:
%%writefile ../src/timerImportTest.js
var _abiJson = require('../src/timerABI.json');
var _binJson = require('../src/timerBIN.json');
contractName = Object.keys(_abiJson.contracts);
console.log("contract name: "+contractName[0]);   // or contractName

_abi = _abiJson.contracts[contractName].abi;
_bin = _binJson.contracts[contractName].bin;
console.log("abi: "+_abi);
console.log("bin: "+_bin);

Overwriting ../src/timerImportTest.js


In [27]:
!node ../src/timerImportTest.js

contract name: ../src/timer.sol:Timer
abi: [{"inputs":[],"name":"getNow","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"start","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"timePassed","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"}]
bin: 608060405234801561001057600080fd5b5060af8061001f6000396000f3fe6080604052348015600f57600080fd5b5060043610603c5760003560e01c8063b4454253146041578063bbe4fd50146059578063be9a655514605f575b600080fd5b60476067565b60408051918252519081900360200190f35b6047606f565b60656073565b005b600054420390565b4290565b4260005556fea26469706673582212201c29482d79d2c4ff810f50d1c89bdfe4eeb9ff22d1b67bc3b4058eaf961fe1c564736f6c63430006040033


## 3. 배포

In [28]:
%%writefile ../src/timerDeploy-ganache.js
var Web3 = require('web3');
var _abiJson = require('../src/timerABI.json');
var _binJson = require('../src/timerBIN.json');

if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
}

contractName = Object.keys(_abiJson.contracts);
console.log("contract name: "+contractName[0]);
var _abi = JSON.parse(_abiJson.contracts[contractName].abi);
var _bin = _binJson.contracts[contractName].bin;
console.log("abi: "+_abi);
console.log("bin: "+_bin);

var _contract = new web3.eth.Contract(_abi);

// unlock the account with a password provided
// web3.personal.unlockAccount(web3.eth.accounts[0], 'password');

_contract
    .deploy({ data: "0x"+_bin })
    .send({
     from: '0x6039EcA4b432697CfA4f787135C1E9641fe10A4F',
     gas: '1000000'
    })
    .then(function(newContractInstance){
        console.log("contract address: "+newContractInstance.options.address)
    });

Writing ../src/timerDeploy-ganache.js


In [29]:
!node ../src/timerDeploy-ganache.js

contract name: ../src/timer.sol:Timer
abi: [object Object],[object Object],[object Object]
bin: 608060405234801561001057600080fd5b5060af8061001f6000396000f3fe6080604052348015600f57600080fd5b5060043610603c5760003560e01c8063b4454253146041578063bbe4fd50146059578063be9a655514605f575b600080fd5b60476067565b60408051918252519081900360200190f35b6047606f565b60656073565b005b600054420390565b4290565b4260005556fea26469706673582212201c29482d79d2c4ff810f50d1c89bdfe4eeb9ff22d1b67bc3b4058eaf961fe1c564736f6c63430006040033
contract address: 0xE340153E27b2Ea837552E83E6d3C59f0411261fD


## 4. 사용

In [44]:
%%writefile ../src/timerUse-ganache.js
var Web3 = require('web3');
var _abiJson = require('../src/timerABI.json');

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName = Object.keys(_abiJson.contracts);
// console.log("contract name: "+contractName[0]);
var _abi = JSON.parse(_abiJson.contracts[contractName].abi);
// console.log("abi: "+_abi);

var timer = new web3.eth.Contract(_abi, "0xE340153E27b2Ea837552E83E6d3C59f0411261fD");
timer.methods.getNow().call().then(function(value) { console.log(value); });
timer.methods.start().send({from: "0x6039EcA4b432697CfA4f787135C1E9641fe10A4F", gas: 1000000});
timer.methods.timePassed().call().then(function(value) { console.log(value); });

Overwriting ../src/timerUse-ganache.js


In [45]:
!node ../src/timerUse-ganache.js

1619106286
0
